# Attributes and Functions

The following shows all of the needed attributes and functions to use the package. These are in the that one would tend to need them to perform a calculation. All values shown here are the default values used. Note the next is not a runnable cell! Scroll down for an actual example. 

```Python
import pbj ## name to use for import

simulation = pbj.electrostatics.Simulation() # Set up simulation

## Creation of solute object which creates and imports surface mesh, and imports charge values and positions
protein = pbj.electrostatics.Solute(
    solute_file_path, ## This is the location of pqr file for the solute
    external_mesh_file = None, ## Used to define location of external pre-create mesh to be used. If defined without file extension (../../surf_mesh) it is assumed to be an msms mesh (.vert/.face), else it is imported based on the file extension and can be of any type available in meshio
    save_mesh_build_files = False, ## Whether you wish to save the files created during the building and import of the mesh (otherwise this takes place in a tempory folder)
    mesh_build_files_dir = "mesh_files/", ## The directory where to save biuld files, if this is requested
    mesh_density = 1.0, ## If msms is used to mesh this is the mesh density, however, if nanoshaper is used this is the grid_scale (Approximate conversions: density of 4 ≈ grid_scale of 1.6, density of 11 ≈ grid_scale of 2.7, density of 23 ≈ grid_scale of 3.9)
    mesh_probe_radius = 1.4, ## Size of the probe to be used during SES generation
    mesh_generator = "nanoshaper", ## Which porgram to use for gerating the surface mesh. NanoShpaer is inclued with the library, 
    print_times = False, ## Whether to print the time taken for varouis parts of the calculation (time to genertate RHS, time to pass to discrete, etc.)
    force_field = "amber", ## Which force field to use in the generation of a pqr from a pdb (pdb2pqr must be installed for this)
    formulation='direct', ## boundary integral formulation (see https://doi.org/10.1002/jcc.26825)
    radius_keyword='solute', ## for Amoeba runs, find atomic radius under 'solute' or 'vdw' keywords
    solute_radius_type='PB' ## for Amoeba runs and radius_keyword=='solute' choose PB, DDCOSMO, or GK optmized radii
)

## Add your protein to the simulation
simulation.add_solute(protein)

## The following attributes of the created object are used to save the arguments specified during the object's creation
simulation.solutes[0].save_mesh_build_files
simulation.solutes[0].mesh_build_files_dir
simulation.solutes[0].mesh_density
simulation.solutes[0].mesh_probe_radius
simulation.solutes[0].mesh_generator
simulation.force_field

## As either a pdb or pqr can be specified, the following attribute shows which was used
simulation.solutes[0].imported_file_type
## If a pdb is used the path is saved to:
simulation.solutes[0].pdb_path
## else the used pqr path is saved to:
simulation.solutes[0].pqr_path

## The solute name is saved to the following attribute. If a pdb is used it is taken from this, otherwise it corresponds to the pqr filename
simulation.solutes[0].solute_name

## simulation.solutes[0].XX points at the same place as protein.XX, so you can also access that way

protein.mesh ## Attribute contining the bemmp mesh object of the solute
protein.q ## Attribute containing the pioint charges present in the the solute
protein.x_q ## Attribute 

protein.mesh.number_of_elements

## There are three different formulations avaible to solve the potential across the boundary
## These are "direct" (linearized Poisson–Boltzmann equation), "juffer" (see Juffer ETal 1991) and "alpha_beta" (currently under analysis)
## The formulation to be used is set with the following attribute:
simulation.pb_formulation = "direct"

## Attributes for the internal and external dielectric constants and inverse Debye length (kappa)
simulation.solutes[0].ep_in = 4.0
simulation.ep_ex = 80.0
simulation.kappa = 0.125

## Attributes for the aplha and beta values to be used in the case that this formulation is used
simulation.pb_formulation_alpha = 1.0
simulation.pb_formulation_beta = self.ep_ex/self.ep_in

## Whether to apply or not calderon preconditioning (only if using alpha_beta), and which to apply (squared, interior or exterior)
simulation.solutes[0].pb_formulation_preconditioning = False
simulation.solutes[0].pb_formulation_preconditioning_type = "block_diagonal"

## Attributes that can be changed for the GMRES solver
simulation.gmres_tolerance = 1e-5
simulation.gmres_max_iterations = 1000
simulation.gmres_restart = 1000

simulation.calculate_potential() ## Function to solve the potential across the boundary using the parameters set above (formulation, dielectric constants, etc.)

## Attributes to whcih the results are saved
protein.results['solver_iteration_count'] ## Iteration count of the GMRES solver
protein.results['phi'] ## Bempp grid function of the potential on the surface mesh
protein.results['d_phi'] ## Bempp grid function of the derivative of the potential on the surface mesh

simulation.calculate_solvation_energy() ## Function to calculate the solvatation energy of the solute, using the results of the surface potential calculation

## Attributes to whcih the results are saved
protein.results['solvation_energy'] ## Solvatation energy calculated, in [kcal/mol]
```

---

# Example

Now a simple example of how to calulate the solvatation energy of a protein using an previously generated pqr file. In this case it will be the 1bpi protein having generated the pqr from the pdb file using the CHARMM force field. First we must import the main package.

In [ ]:
import pbj

Now we create solute object, giving the path of the pqr file to be used.

In [ ]:
simulation = pbj.electrostatics.Simulation()
protein = pbj.electrostatics.Solute("pqrs/1bpi.pqr")
simulation.add_solute(protein)

If a previously created msms or other type of mesh is to be used, we can import it as follows.

In [ ]:
#protein = pbj.electrostatics.Solute("5pti.pqr" , external_mesh_file = "surf_d02") ## If no extension is found it is assumed to be .vert/.face

#protein = pbj.electrostatics.Solute("5pti.pqr" , external_mesh_file = "surf_d02.off") ## Or any file supported by meshio can be used

Now we have gerated and imported the mesh, as well as the charges and their positons. As the mesh is a bempp mesh object we can manipulate it with the corresponding functions for this object.

In [ ]:
print("Number of elements:", protein.mesh.number_of_elements)
print("Number of vertices:", protein.mesh.number_of_vertices)
#protein.mesh.plot() ## Uncomment to see a plot of the surface mesh

As we will calculate the energy using all of the default options we only need call the calculate_solvation_energy() function.

In [ ]:
simulation.calculate_solvation_energy()

This has first calculated the potential on the surface mesh, then calculated the solvation energy of the protein which we can print as follows:

In [ ]:
print("Solvation energy:", simulation.solutes[0].results['solvation_energy'], "[kcal/mol]")

Let's check out what other intermediate results are available after computing the solvation energy

In [ ]:
print(simulation.solutes[0].results.keys())

Now we can, for example, plot the potential on the surface 

In [ ]:
simulation.solutes[0].results['phi'].plot()

Let's rerun with a different formulation, such as the one used by Lu et al (see https://doi.org/10.1073/pnas.0605166103 and https://doi.org/10.1002/jcc.26825)

In [ ]:
simulation.solutes[0].display_available_formulations()

In [ ]:
simulation2 = pbj.electrostatics.Simulation(formulation='lu')
protein2 = pbj.electrostatics.Solute("pqrs/1bpi.pqr")
simulation2.add_solute(protein2)

In [ ]:
simulation2.calculate_solvation_energy()

In [ ]:
print("Solvation energy:", simulation2.solutes[0].results['solvation_energy'], "[kcal/mol]")

In [ ]:
print(simulation.timings)
print(simulation2.timings)